In [1]:
import torchvision
import torchvision.transforms as transforms
import os
import time
from SCNN_Layer import*
from Iterative_LIF_Model import *

In [2]:
names = 'STBPmodel'
data_path =  '../../../DATA/' #todo: input your data path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
train_dataset = torchvision.datasets.MNIST(root= data_path, train=True, download=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [5]:
test_set = torchvision.datasets.MNIST(root= data_path, train=False, download=False,  transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

In [5]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
acc_record = []
loss_train_record = []
loss_test_record = []

In [6]:
snn = SCNN()
snn.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(snn.parameters(), lr=learning_rate)

In [7]:
num_epochs = 100 # max epoch
num_classes = 10

In [8]:
for epoch in range(num_epochs):
    train_loss = 0
    start_time = time.time()
    for i, (train_images, train_labels) in enumerate(train_loader):#共计600个batch，i=0-599
        snn.zero_grad()
        optimizer.zero_grad()
        
        train_images = train_images.float().to(device)
        train_predicts = snn(train_images)
        train_labels = torch.zeros(batch_size, num_classes).scatter_(1, train_labels.view(-1,1), 1.)
                                            #将index视作列（dim = 1），按照train_labels.view(-1,1)值作为index，把src =“1”插入到zeros中
        loss = criterion(train_predicts.cpu(), train_labels)
        train_loss = loss.item() + train_loss
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:#每100个batch报告一次，即100*100张图片报告一次
            print('Epoch [%d/%d], Train Step [%d/%d], Train Loss = %.5f' 
                    %(epoch + 1, num_epochs, i + 1, len(train_dataset)//batch_size, train_loss))
            train_loss = 0
            print('Time elasped:', time.time() - start_time)
    correct = 0
    total = 0
    optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)
    
    with torch.no_grad():
        for j, (test_images, test_targets) in enumerate(test_loader):
            optimizer.zero_grad()
            snn.zero_grad()
            
            test_images = test_images.float().to(device)
            test_predicts = snn(test_images)
            test_labels = torch.zeros(batch_size, num_classes).scatter_(1, test_targets.view(-1,1), 1.)
            loss = criterion(test_predicts.cpu(), test_labels)
            
            _, predicted = test_predicts.cpu().max(1)
            total = total + float(test_targets.size(0))
            correct = correct + float(predicted.eq(test_targets).sum().item())
            if (j + 1) % 100 == 0:
                acc = 100. * float(correct)/float(total)
                print('Test Step [%d/%d], Acc: %.5f' %(j + 1, len(test_loader), acc))
    acc = float(100 * correct / total)
    print('Epoch [%d/%d] \t Test Accuracy Over Test Dataset: %.3f' %(epoch + 1, num_epochs, acc))
    acc_record.append(acc)
    if epoch % 5 == 0:
        print('Saving......')
        state = {
            'net': snn.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'acc_record': acc_record,
        }
        if not os.path.isdir('./history'):
            os.mkdir('./history')
        torch.save(state, './history/' + names + '.pk')

Epoch [1/100], Train Step [100/600], Train Loss = 4.31109
Time elasped: 9.958385229110718
Epoch [1/100], Train Step [200/600], Train Loss = 0.95825
Time elasped: 18.380383253097534
Epoch [1/100], Train Step [300/600], Train Loss = 0.65087
Time elasped: 26.807471752166748
Epoch [1/100], Train Step [400/600], Train Loss = 0.46130
Time elasped: 35.25646424293518
Epoch [1/100], Train Step [500/600], Train Loss = 0.36865
Time elasped: 43.69429349899292
Epoch [1/100], Train Step [600/600], Train Loss = 0.36949
Time elasped: 52.157328605651855
Test Step [100/100], Acc: 98.47000
Epoch [1/100] 	 Test Accuracy Over Test Dataset: 98.470
Saving......
Epoch [2/100], Train Step [100/600], Train Loss = 0.31679
Time elasped: 8.456356287002563
Epoch [2/100], Train Step [200/600], Train Loss = 0.26933
Time elasped: 16.90826988220215
Epoch [2/100], Train Step [300/600], Train Loss = 0.27841
Time elasped: 25.40502429008484
Epoch [2/100], Train Step [400/600], Train Loss = 0.29371
Time elasped: 33.88785076

Test Step [100/100], Acc: 99.19000
Epoch [13/100] 	 Test Accuracy Over Test Dataset: 99.190
Epoch [14/100], Train Step [100/600], Train Loss = 0.03981
Time elasped: 8.504587650299072
Epoch [14/100], Train Step [200/600], Train Loss = 0.04575
Time elasped: 17.001006841659546
Epoch [14/100], Train Step [300/600], Train Loss = 0.03665
Time elasped: 25.47086524963379
Epoch [14/100], Train Step [400/600], Train Loss = 0.02769
Time elasped: 33.9811327457428
Epoch [14/100], Train Step [500/600], Train Loss = 0.04544
Time elasped: 42.47619342803955
Epoch [14/100], Train Step [600/600], Train Loss = 0.04783
Time elasped: 50.99836206436157
Test Step [100/100], Acc: 99.36000
Epoch [14/100] 	 Test Accuracy Over Test Dataset: 99.360
Epoch [15/100], Train Step [100/600], Train Loss = 0.03170
Time elasped: 8.514951705932617
Epoch [15/100], Train Step [200/600], Train Loss = 0.02661
Time elasped: 17.03672981262207
Epoch [15/100], Train Step [300/600], Train Loss = 0.03585
Time elasped: 25.537678480148

Epoch [26/100], Train Step [600/600], Train Loss = 0.02251
Time elasped: 50.98703694343567
Test Step [100/100], Acc: 99.30000
Epoch [26/100] 	 Test Accuracy Over Test Dataset: 99.300
Saving......
Epoch [27/100], Train Step [100/600], Train Loss = 0.01081
Time elasped: 8.473367929458618
Epoch [27/100], Train Step [200/600], Train Loss = 0.01690
Time elasped: 16.99891495704651
Epoch [27/100], Train Step [300/600], Train Loss = 0.01480
Time elasped: 25.5408296585083
Epoch [27/100], Train Step [400/600], Train Loss = 0.01382
Time elasped: 34.00385928153992
Epoch [27/100], Train Step [500/600], Train Loss = 0.01993
Time elasped: 42.511406898498535
Epoch [27/100], Train Step [600/600], Train Loss = 0.01573
Time elasped: 50.9819438457489
Test Step [100/100], Acc: 99.29000
Epoch [27/100] 	 Test Accuracy Over Test Dataset: 99.290
Epoch [28/100], Train Step [100/600], Train Loss = 0.01204
Time elasped: 8.459954977035522
Epoch [28/100], Train Step [200/600], Train Loss = 0.01329
Time elasped: 16.

Epoch [39/100], Train Step [500/600], Train Loss = 0.01076
Time elasped: 42.44963002204895
Epoch [39/100], Train Step [600/600], Train Loss = 0.00702
Time elasped: 50.94272708892822
Test Step [100/100], Acc: 99.32000
Epoch [39/100] 	 Test Accuracy Over Test Dataset: 99.320
Epoch [40/100], Train Step [100/600], Train Loss = 0.01111
Time elasped: 8.461677074432373
Epoch [40/100], Train Step [200/600], Train Loss = 0.00998
Time elasped: 16.958486795425415
Epoch [40/100], Train Step [300/600], Train Loss = 0.00906
Time elasped: 25.427454233169556
Epoch [40/100], Train Step [400/600], Train Loss = 0.01031
Time elasped: 33.92059803009033
Epoch [40/100], Train Step [500/600], Train Loss = 0.00904
Time elasped: 42.42454528808594
Epoch [40/100], Train Step [600/600], Train Loss = 0.00794
Time elasped: 50.923415660858154
Test Step [100/100], Acc: 99.40000
Epoch [40/100] 	 Test Accuracy Over Test Dataset: 99.400
Epoch [41/100], Train Step [100/600], Train Loss = 0.00816
Time elasped: 8.4797532558

Epoch [52/100], Train Step [400/600], Train Loss = 0.00288
Time elasped: 33.990123987197876
Epoch [52/100], Train Step [500/600], Train Loss = 0.00386
Time elasped: 42.49362134933472
Epoch [52/100], Train Step [600/600], Train Loss = 0.00338
Time elasped: 51.01633429527283
Test Step [100/100], Acc: 99.44000
Epoch [52/100] 	 Test Accuracy Over Test Dataset: 99.440
Epoch [53/100], Train Step [100/600], Train Loss = 0.00360
Time elasped: 8.47862720489502
Epoch [53/100], Train Step [200/600], Train Loss = 0.00146
Time elasped: 16.94634175300598
Epoch [53/100], Train Step [300/600], Train Loss = 0.00475
Time elasped: 25.427849292755127
Epoch [53/100], Train Step [400/600], Train Loss = 0.00478
Time elasped: 33.8937566280365
Epoch [53/100], Train Step [500/600], Train Loss = 0.00578
Time elasped: 42.37756896018982
Epoch [53/100], Train Step [600/600], Train Loss = 0.00241
Time elasped: 50.841418981552124
Test Step [100/100], Acc: 99.37000
Epoch [53/100] 	 Test Accuracy Over Test Dataset: 99.

Epoch [65/100], Train Step [300/600], Train Loss = 0.00342
Time elasped: 25.68175435066223
Epoch [65/100], Train Step [400/600], Train Loss = 0.00326
Time elasped: 34.2637836933136
Epoch [65/100], Train Step [500/600], Train Loss = 0.00537
Time elasped: 42.890578746795654
Epoch [65/100], Train Step [600/600], Train Loss = 0.00481
Time elasped: 51.70103621482849
Test Step [100/100], Acc: 99.44000
Epoch [65/100] 	 Test Accuracy Over Test Dataset: 99.440
Epoch [66/100], Train Step [100/600], Train Loss = 0.00754
Time elasped: 8.835338115692139
Epoch [66/100], Train Step [200/600], Train Loss = 0.00125
Time elasped: 17.616270065307617
Epoch [66/100], Train Step [300/600], Train Loss = 0.00355
Time elasped: 26.37375783920288
Epoch [66/100], Train Step [400/600], Train Loss = 0.00436
Time elasped: 34.91354560852051
Epoch [66/100], Train Step [500/600], Train Loss = 0.00151
Time elasped: 43.422356605529785
Epoch [66/100], Train Step [600/600], Train Loss = 0.00222
Time elasped: 51.92280650138

Epoch [78/100], Train Step [200/600], Train Loss = 0.00561
Time elasped: 17.043738842010498
Epoch [78/100], Train Step [300/600], Train Loss = 0.00396
Time elasped: 25.534474849700928
Epoch [78/100], Train Step [400/600], Train Loss = 0.00230
Time elasped: 34.01510310173035
Epoch [78/100], Train Step [500/600], Train Loss = 0.00335
Time elasped: 42.51297450065613
Epoch [78/100], Train Step [600/600], Train Loss = 0.00155
Time elasped: 50.98888897895813
Test Step [100/100], Acc: 99.46000
Epoch [78/100] 	 Test Accuracy Over Test Dataset: 99.460
Epoch [79/100], Train Step [100/600], Train Loss = 0.00632
Time elasped: 8.463370084762573
Epoch [79/100], Train Step [200/600], Train Loss = 0.00137
Time elasped: 16.944433212280273
Epoch [79/100], Train Step [300/600], Train Loss = 0.00412
Time elasped: 25.405837059020996
Epoch [79/100], Train Step [400/600], Train Loss = 0.00121
Time elasped: 33.88513517379761
Epoch [79/100], Train Step [500/600], Train Loss = 0.00248
Time elasped: 42.355789661

Epoch [91/100], Train Step [100/600], Train Loss = 0.00268
Time elasped: 8.524151802062988
Epoch [91/100], Train Step [200/600], Train Loss = 0.00110
Time elasped: 17.032883405685425
Epoch [91/100], Train Step [300/600], Train Loss = 0.00538
Time elasped: 25.575085163116455
Epoch [91/100], Train Step [400/600], Train Loss = 0.00221
Time elasped: 34.08192205429077
Epoch [91/100], Train Step [500/600], Train Loss = 0.00410
Time elasped: 42.63657522201538
Epoch [91/100], Train Step [600/600], Train Loss = 0.00202
Time elasped: 51.152703285217285
Test Step [100/100], Acc: 99.41000
Epoch [91/100] 	 Test Accuracy Over Test Dataset: 99.410
Saving......
Epoch [92/100], Train Step [100/600], Train Loss = 0.00217
Time elasped: 8.681618213653564
Epoch [92/100], Train Step [200/600], Train Loss = 0.00511
Time elasped: 17.201967477798462
Epoch [92/100], Train Step [300/600], Train Loss = 0.00214
Time elasped: 25.718971252441406
Epoch [92/100], Train Step [400/600], Train Loss = 0.00414
Time elasped